In [1]:
# make local files accessible to import statements
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../..'))

In [2]:
from numpy import errstate
from PySDM_examples.Shima_et_al_2009_Fig_2.plotter import Plotter
from PySDM_examples.Shima_et_al_2009_Fig_2.setup import SetupA
from PySDM_examples.Shima_et_al_2009_Fig_2.example import run
import ipywidgets as widgets
%matplotlib notebook

In [3]:
progbar = widgets.IntProgress(min=0, max=100, description='%')
def update_progbar(value):
    progbar.value = 100 * value

In [4]:
def demo(*, n_SD, n_step, n_plot):
    with errstate(all='raise'):
        setup = SetupA()
        
        setup.n_sd = 2 ** n_SD
        setup.steps = [i * (n_step // n_plot) for i in range(n_plot + 1)]

        setup.check = lambda _, step: update_progbar(step / setup.steps[-1])
        
        states, _ = run(setup)
    
        x_min = min([state.min('volume') for state in states.values()])
        x_max = max([state.max('volume') for state in states.values()])

    with errstate(invalid='ignore'):
        plotter = Plotter(setup, (x_min, x_max))
        for step, state in states.items():
            plotter.plot(state, step * setup.dt)
        plotter.show()

In [5]:
n_SD = widgets.IntSlider(value=14, min=12, max=18, step=1, description='log_2(n_SD)', continuous_update=False)
n_step = widgets.IntSlider(value=3600, step=100, min=0, max=3600, description='n_step', continuous_update=False)
n_plot = widgets.IntSlider(value=3, step=1, min=1, max=8, description='n_plot', continuous_update=False)
ui = widgets.HBox([n_SD, n_step, n_plot])

self = widgets.interactive_output(demo, {'n_SD': n_SD, 'n_step': n_step, 'n_plot': n_plot})

display(ui, progbar, self)